In [ ]:
!pip install -q datasets pandas sentencepiece transformers

In [ ]:
import pandas as pd
import os
import shutil
from datasets import load_dataset
from transformers import AutoConfig

In [ ]:
base_model_name = "naver-clova-ix/donut-base-finetuned-rvlcdip"
csv_path ="/content/drive/MyDrive/donutData/fullCsv.csv"
# Set to True to reset image folder and make new training and test dataset
reset_ImageFolder = False
#parent folder of imageFolder
train_data_path_parent = r'imageFolder'
dataset_dict_folder = 'datasetDict'

In [ ]:
df = pd.read_csv(csv_path).sample(frac=1).reset_index(drop=True)

display(df.head())


parent_folder = '/content/drive/MyDrive/donutData/fullData2'


df['image'] = df['image'].apply(lambda x: os.path.join(parent_folder,x).replace('\\', '/'))
df['ground_truth'] = df['label'].apply(lambda x: {"gt_parse": {"class": x}})


display(df.head())

,image,label
0,dataset\autoDownloadedDataset\huggingData\sing...,tax_form
1,dataset\autoDownloadedDataset\huggingData\sing...,tax_form
2,dataset\autoDownloadedDataset\huggingData\deep...,passport
3,dataset\autoDownloadedDataset\huggingData\sing...,tax_form
4,dataset\oraganized\passport\da04c641110ab3c6af...,passport


,image,label,ground_truth
0,/content/drive/MyDrive/donutData/fullData2/dat...,tax_form,{'gt_parse': {'class': 'tax_form'}}
1,/content/drive/MyDrive/donutData/fullData2/dat...,tax_form,{'gt_parse': {'class': 'tax_form'}}
2,/content/drive/MyDrive/donutData/fullData2/dat...,passport,{'gt_parse': {'class': 'passport'}}
3,/content/drive/MyDrive/donutData/fullData2/dat...,tax_form,{'gt_parse': {'class': 'tax_form'}}
4,/content/drive/MyDrive/donutData/fullData2/dat...,passport,{'gt_parse': {'class': 'passport'}}


In [ ]:
label_distribution = df['label'].value_counts()
print(label_distribution)


label
passport            4342
tax_form            2000
driving_license      970
income_statement     317
balance_sheets       282
invoice              199
bank_statement       149
cash_flow             36
Name: count, dtype: int64


In [ ]:
filtered_df = pd.concat([
    df[df['label'] == 'passport'].sample(1000, random_state=42),
    df[df['label'] == 'tax_form'].sample(1000, random_state=42),
    df[~df['label'].isin(['passport', 'tax_form'])]
])

# Reset the index for a cleaner DataFrame
filtered_df = filtered_df.reset_index(drop=True)

# Display the new label distribution
print(filtered_df['label'].value_counts())

df = filtered_df
print(df)

label
passport            1000
tax_form            1000
driving_license      970
income_statement     317
balance_sheets       282
invoice              199
bank_statement       149
cash_flow             36
Name: count, dtype: int64
                                                  image            label  \
0     /content/drive/MyDrive/donutData/fullData2/dat...         passport   
1     /content/drive/MyDrive/donutData/fullData2/dat...         passport   
2     /content/drive/MyDrive/donutData/fullData2/dat...         passport   
3     /content/drive/MyDrive/donutData/fullData2/dat...         passport   
4     /content/drive/MyDrive/donutData/fullData2/dat...         passport   
...                                                 ...              ...   
3948  /content/drive/MyDrive/donutData/fullData2/dat...  driving_license   
3949  /content/drive/MyDrive/donutData/fullData2/dat...  driving_license   
3950  /content/drive/MyDrive/donutData/fullData2/dat...   balance_sheets   
3951  /c

In [ ]:
def simpleTest(img_path):
  return os.path.exists(img_path)


first_row = df.iloc[0]

print("did simple file exist pass: ", simpleTest(first_row['image']))
print(first_row['image'])

did simple file exist pass:  True
/content/drive/MyDrive/donutData/fullData2/dataset/oraganized/passport/IMG_20240806_153441_jpg.rf.1ca18a4fff53e5a4512ef0504029249e.jpg


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split



# Dataset paths
train_path = os.path.join(train_data_path_parent, 'train')
test_path = os.path.join(train_data_path_parent, 'test')



# Removing old folders if reset is needed
if reset_ImageFolder and (os.path.exists(train_path) or os.path.exists(test_path)):
    if os.path.exists(train_path):
        shutil.rmtree(train_path)
    if os.path.exists(test_path):
        shutil.rmtree(test_path)

# Create folders if they do not exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Split the dataframe into train and test sets (80% train, 20% test by default)
# Use stratify to maintain the class distribution
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Function to copy files to respective directories
def copy_images(df, destination_folder):
    for index, row in df.iterrows():
        label_path = os.path.join(destination_folder, row['label'])
        # Create label folder if it doesn't exist
        if not os.path.exists(label_path):
            os.makedirs(label_path)
        # Copy the image to the destination
        shutil.copy(row['image'], os.path.join(label_path, os.path.basename(row['image'])))

# Check if reset is true or folders are empty, and then copy images
if reset_ImageFolder or not (os.listdir(train_path) and os.listdir(test_path)):
    copy_images(train_df, train_path)
    copy_images(test_df, test_path)

print('Image folder split completed.')
print('Train folder in:', train_path)
print('Test folder in:', test_path)


Image folder split completed.
Train folder in: imageFolder/train
Test folder in: imageFolder/test


In [ ]:
dataset_dict = load_dataset("imagefolder", data_dir=train_data_path_parent)



Resolving data files:   0%|          | 0/3162 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/791 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_dict['train'][6]



{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x606>,
 'label': 0}

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3162
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 791
    })
})

In [ ]:
dataset_dict['train'][:5]

{'image': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1195>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1216>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1182>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1217>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1365>],
 'label': [0, 0, 0, 0, 0]}

In [ ]:
id2label_test = {id: label for id, label in enumerate(dataset_dict['train'].features['label'].names)}
print(id2label_test)

{0: 'balance_sheets', 1: 'bank_statement', 2: 'cash_flow', 3: 'driving_license', 4: 'income_statement', 5: 'invoice', 6: 'passport', 7: 'tax_form'}


In [ ]:
new_labels = set(df['label'].unique())

old_id2label = {
  0: "letter",
  1: "form",
  2: "email",
  3: "handwritten",
  4: "advertisement",
  5: "scientific_report",
  6: "scientific_publication",
  7: "specification",
  8: "file_folder",
  9: "news_article",
  10: "budget",
  11: "invoice",
  12: "presentation",
  13: "questionnaire",
  14: "resume",
  15: "memo",

}

old_labels = set(old_id2label.values())

unique_new_labels = new_labels - old_labels

max_old_id = max(old_id2label.keys())
new_id2label = old_id2label.copy()

# Adding new labels to the dictionary
for idx, label in enumerate(unique_new_labels, start=max_old_id + 1):
    new_id2label[idx] = label

# Display the updated new_id2label dictionary
new_id2label

{0: 'letter',
 1: 'form',
 2: 'email',
 3: 'handwritten',
 4: 'advertisement',
 5: 'scientific_report',
 6: 'scientific_publication',
 7: 'specification',
 8: 'file_folder',
 9: 'news_article',
 10: 'budget',
 11: 'invoice',
 12: 'presentation',
 13: 'questionnaire',
 14: 'resume',
 15: 'memo',
 16: 'cash_flow',
 17: 'passport',
 18: 'driving_license',
 19: 'balance_sheets',
 20: 'bank_statement',
 21: 'tax_form',
 22: 'income_statement'}

In [ ]:
from datasets import ClassLabel, DatasetDict

# Assuming 'dataset_dict' is your DatasetDict instance
# Get the existing features of the train split (assuming train split has representative features)
features = dataset_dict['train'].features

# Replace the label feature with a new ClassLabel with the desired number of classes
new_num_classes = len(new_id2label)  # change to new id length
features['label'] = ClassLabel(num_classes=new_num_classes)

# Recreate the datasets with the updated features
updated_dataset_dict = DatasetDict({
    split: dataset.cast(features) for split, dataset in dataset_dict.items()
})

Casting the dataset:   0%|          | 0/3162 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/791 [00:00<?, ? examples/s]

In [ ]:
# Assuming dataset_dict is your DatasetDict instance
# Let's examine the 'train' split to retrieve the label mapping

# Access the features of the 'train' split
features = dataset_dict['train'].features

# Check if 'label' feature is of ClassLabel type and get the mapping if so
if isinstance(features['label'], ClassLabel):
    curr_id2label = {i: features['label'].int2str(i) for i in range(features['label'].num_classes)}
    print(curr_id2label)

else:
    print("The 'label' feature is not of ClassLabel type, cannot extract ID-to-label mapping.")




{0: 'balance_sheets', 1: 'bank_statement', 2: 'cash_flow', 3: 'driving_license', 4: 'income_statement', 5: 'invoice', 6: 'passport', 7: 'tax_form'}


In [ ]:
new_label2id = {value: key for key, value in new_id2label.items()}
new_label2id

{'letter': 0,
 'form': 1,
 'email': 2,
 'handwritten': 3,
 'advertisement': 4,
 'scientific_report': 5,
 'scientific_publication': 6,
 'specification': 7,
 'file_folder': 8,
 'news_article': 9,
 'budget': 10,
 'invoice': 11,
 'presentation': 12,
 'questionnaire': 13,
 'resume': 14,
 'memo': 15,
 'cash_flow': 16,
 'passport': 17,
 'driving_license': 18,
 'balance_sheets': 19,
 'bank_statement': 20,
 'tax_form': 21,
 'income_statement': 22}

In [ ]:
from datasets import DatasetDict



def update_label(example):

  currId = example['label']
  str_label = curr_id2label[currId]
  newId = new_label2id[str_label]
  example['label'] = newId
  return example

updated_dataset_dict = DatasetDict({
    split: dataset.map(update_label) for split, dataset in updated_dataset_dict.items()
})


# Update the label feature in the dataset's features
new_class_label = ClassLabel(names=list(new_label2id.keys()))

# Update the features to use the new label mapping
for split in updated_dataset_dict:
    updated_dataset_dict[split] = updated_dataset_dict[split].cast_column('label', new_class_label)





Map:   0%|          | 0/3162 [00:00<?, ? examples/s]

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3162 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/791 [00:00<?, ? examples/s]

In [ ]:
id2label_test = {id: label for id, label in enumerate(updated_dataset_dict['train'].features['label'].names)}
print(id2label_test)

{0: 'letter', 1: 'form', 2: 'email', 3: 'handwritten', 4: 'advertisement', 5: 'scientific_report', 6: 'scientific_publication', 7: 'specification', 8: 'file_folder', 9: 'news_article', 10: 'budget', 11: 'invoice', 12: 'presentation', 13: 'questionnaire', 14: 'resume', 15: 'memo', 16: 'cash_flow', 17: 'passport', 18: 'driving_license', 19: 'balance_sheets', 20: 'bank_statement', 21: 'tax_form', 22: 'income_statement'}


In [ ]:
# Extract the first 5 rows
first_five_rows = updated_dataset_dict['train'][:5]

# Print the actual data for the first 5 rows
print(first_five_rows)

# Extract the first 5 rows
first_five_rows = updated_dataset_dict['test'][:5]

# Print the actual data for the first 5 rows
print(first_five_rows)


{'image': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1195 at 0x7F01046F7A00>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1216 at 0x7F01046F4940>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1182 at 0x7F01046F4E80>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1217 at 0x7F01046F7F70>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1365 at 0x7F01A1F700D0>], 'label': [19, 19, 19, 19, 19]}
{'image': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1048 at 0x7F01A1F73070>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x571 at 0x7F01A1F73880>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1191 at 0x7F01A1F72740>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x546 at 0x7F01A1F71660>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=819x1037 at 0x7F01A1F710C0>], 'label': [19, 19, 19, 19, 19]}


Prepare to Donut Dataset


In [ ]:
template = '{"gt_parse": {"class" : '

In [ ]:

id2label = new_id2label


def update_examples(examples):
  ground_truths = []
  for label in examples['label']:
    ground_truths.append(template + '"' + id2label[label] + '"' + "}}")

  examples['ground_truth'] = ground_truths

  return examples

donut_dataset = updated_dataset_dict.map(update_examples, batched=True)





Map:   0%|          | 0/3162 [00:00<?, ? examples/s]

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

In [ ]:
test = donut_dataset['train'][500]['ground_truth']
test

'{"gt_parse": {"class" : "driving_license"}}'

In [ ]:

from ast import literal_eval

test2 = literal_eval(test)
test2['gt_parse']

{'class': 'driving_license'}

In [ ]:
donut_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'ground_truth'],
        num_rows: 3162
    })
    test: Dataset({
        features: ['image', 'label', 'ground_truth'],
        num_rows: 791
    })
})

In [ ]:
donut_dataset.save_to_disk(dataset_dict_folder)

Saving the dataset (0/3 shards):   0%|          | 0/3162 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/791 [00:00<?, ? examples/s]

In [ ]:
print('lastCell')

lastCell
